# 目標
## 機能
- OpenAIのAPIを使うことなく、LLMを活用する
- LLM活用
  - 任意の文章に対する質疑応答システム (学習済みモデルの活用
  - 独自のデータセット（命令データセット）でのFine-Tuning
  - RLHFの実装と学習


## カスタムLLMを使うメリット
- APIに依存しない（情報を外部に投げない、依存性を排除できる）
- 独自Fine-Tuningできる
- RLHFが実装できる

## テスト課題
- 有価証券報告書に対する質疑応答
  - 課題を質問して課題に近い会社を取ってくる
    - 例) 退職者が多い課題を抱えている会社名を教えてくださいなど

# TaskList
- OpenAIのAPIを使った質疑応答システム(InProgress)  
　  - [LangChain](https://python.langchain.com/en/latest/use_cases/question_answering.html)
    - OpenAI API
- 脱OpenAI
- aaa



In [1]:
!pip install langchain
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install colab-env --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for colab-env: filename=colab_env-0.2.0-py3-none-any.whl size=3829 sha256=5dbc62e48c36e8d1e25be1c2b38cd001312e65676cec3838fcc3b1d8d8986139
  Stored in directory: /root/.cache/pip/wheels/e1/25/d4/95a3d7b269bddc5c6b91e2995e446c01cb03e6472fc5c61507
Successfully built colab-env
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.0
    Uninstalling python-dotenv-1.0.0:
      Successfully uninstalled python-dotenv-1.0.0


Google Driveにvars.envを作成し、OPENAI_API_KEYを設定する

In [5]:
import colab_env

Mounted at /content/gdrive


# ダイキン工業の2023年2月の報告書をダウンロードして配置する
https://disclosure2.edinet-fsa.go.jp/WEEE0030.aspx?bXVsPSVFMyU4MyU4MCVFMyU4MiVBNCVFMyU4MiVBRCVFMyU4MyVCMyZjdGY9b2ZmJmZscz1vbiZscHI9b2ZmJnJwcj1vZmYmb3RoPW9mZiZ5ZXI9Jm1vbj0mcGZzPTYmc2VyPTEmcGFnPTEmc29yPTI=

PDFがそのままだと文字化けして読めなかったので、いったんコピペしてtextにした
ToDo
XMLやPDFからテキスト、テーブルの構造化データを読める形に変換する

In [ ]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader

# loader = UnstructuredFileLoader("/content/test.txt", mode="elements")
loader = UnstructuredFileLoader("/content/test.txt")
docs = loader.load()
docs[0].page_content[:400]

'EDINET提出書類\n\nダイキン工業株式会社(E01570)\n\n四半期報告書\n\n【表紙】\n\n【提出書類】四半期報告書\n\n【根拠条文】金融商品取引法第24条の４の７第１項\n\n【提出先】関東財務局長\n\n【提出日】2023年２月８日\n\n【四半期会計期間】第120期第３四半期(自\n\n【会社名】ダイキン工業株式会社\n\n【英訳名】DAIKIN INDUSTRIES,LTD.\n\n【代表者の役職氏名】取締役社長\n\n【本店の所在の場所】大阪市北区梅田一丁目13番１号大阪梅田ツインタワーズ・サウス\n\n十\n\n河\n\n2022年10月１日\n\n政\n\n至\n\n2022年12月31日)\n\n則\n\n(2022年11月24日より本店所在地\n\n大阪市北区中崎西二丁目４番12号梅田セン\n\nタービルが上記のように移転しております。)\n\n【電話番号】大阪(06)6147-6864\n\n【事務連絡者氏名】経理財務本部経理グループ長\n\n【最寄'

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # separator = "．",  # セパレータ # ToDo別のやつのほうがいい
    chunk_size = 256,  # チャンクの文字数
    chunk_overlap = 0,  # チャンクオーバーラップの文字数
)

In [ ]:
with open('/content/test.txt', 'rt') as f:
    text = f.read()
    texts = text_splitter.split_text(text)

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA

db = Chroma.from_texts(texts, embeddings)
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=db)

query = "今季の売上を教えてください"
qa.run(query)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


DEBUG:Chroma:Index not found
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:Index saved to .chroma/index/index.bin
DEBUG:Chroma:time to pre process our knn query: 4.291534423828125e-06
DEBUG:Chroma:time to run knn query: 0.0005064010620117188


ValidationError: ignored

In [ ]:
query = "今季の売上を教えてください"
index.query(query)

DEBUG:Chroma:time to pre process our knn query: 3.0994415283203125e-06
DEBUG:Chroma:time to run knn query: 0.00022602081298828125


InvalidRequestError: ignored

# 2023-04-12


In [6]:
!wc -l "/content/有価証券2021年度_経営方針と事業課題.csv"

4232 /content/有価証券2021年度_経営方針と事業課題.csv


In [7]:
!head -n 2 "/content/有価証券2021年度_経営方針と事業課題.csv"

﻿,Unnamed: 0,会社名,書類名,docID,証券コード,日付,経営方針,事業課題
0,0,ピープル株式会社,有価証券報告書－第44期(令和2年1月21日－令和3年1月20日),S100L650,78650.0,2021-04-14,１ 【経営方針、経営環境及び対処すべき課題等】文中の将来に関する事項は、当連結会計年度末現在において、当社グループが判断したものです。（1）会社の経営の基本方針及び目標とする経営指標等（経営方針）・ミッション （Peopleの果たすべき社会的役割）子育て生活を研究し、新しい「ためになる」をつくる。・ヴィジョン （なりたい姿）一番身近で「私達（＝子育て生活者）のことを、本当にわかってくれている」商品・サービスを、最前線で世の中に提供し続ける。・バリュー （社員が共有する価値観）1. 最終的な価値を決めるのは消費者時代とともにどんどん変わっていく消費者と真摯に向き合い続ける。2. 市場の主流・最大より、新しい挑戦既存商品からの差別化にとどまらず、独自の戦略で新しいモノ・コト・トキを生み出す。3. 個性を集めた少数精鋭多様な知見を持ち寄ることで、誰も考えつかなかった新しいアイディアが生まれてくる。4. 欺かない・貪らない・侮らない（経営指標）売上営業利益率10％を、上場来、目標として掲げております。（2）事業別課題（第44期（2021年1月期）の振り返り）１．国内事業の業績アップ①ニーズ収集機能の強化ママモニターの増員、その募集範囲の多様化、モニター管理システムのIT化、コロナ禍でもテストできる環境整備など情報収集の質・量ともに大きく前進しました。②企画・開発力の強化より挑戦がしやすい仕組みに改善しました。企画・開発・販売のバリューチェーンをよりスムーズにつなげ、確実にお客様へお届けできる体制に組織変更を行いました。また当社の企画開発の強みを改めて見つめ直し、暗黙知を形式知に変えるためにワークショップなどをパートナー企業の協力のもとに行いました。プロジェクトチームメンバーの認識を共有したことで、よりスピーディーに本質的な議論を行い、より良い商品開発につながるものと期待しています。③プロモーション、販売力の強化コロナ禍にあって、店頭イベントなどリアルな消費者接触が不可能な中で、ウェブ広告を中心に、これまでの当社には無かった新しいプロモー

In [8]:
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.3 MB/s eta 0:00:00


In [9]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/有価証券2021年度_経営方針と事業課題.csv')

data = loader.load()

In [10]:
len(data)

4231

In [11]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

embeddings = HuggingFaceEmbeddings() # "text-embedding-ada-002"
# 脱OpenAIするならここを切り替える

db = Chroma.from_documents(
    data, embeddings, persist_directory='db'
)

1. 入力文章をchunkに区切る (1chunk = 1000 tokenぐらいが良い(はず) ※CSVLoader
2. ベクトル化する (OpenAIEmbeddings)
3. DBにいれる (Chroma.from_documents)
4. プロンプトから類似のchunkを検索する (db.similarity_search)
5. プロンプトと類似のchunkから返答を作成する (chain.run(input_documents=docs, question=query)

In [17]:
query = "退職者が多い課題を抱えている会社名を教えてください"
docs = db.similarity_search(query, k=1)

In [18]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

llm = OpenAI() # text-davinci-003

chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")
chain.run(input_documents=docs, question=query)

InvalidRequestError: ignored

In [14]:
query = "退職者が多い課題を抱えている会社名を教えてください"
index.query_with_sources(query)

NameError: ignored